In [45]:
#import packages and create date

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import *
import shutil
import difflib
from pulp import *
import openpyxl

today = date.today()
#today = today - datetime.timedelta(days=1)
today = today.strftime("%m.%d.20%y")
today

'02.08.2022'

In [46]:
## DKNG Player Pool

contest = "PlayerPool_"
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
contest_data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool'

shutil.move(downloads + "/"+ "DKSalaries.csv",
           contest_data_loc + "/"+ "DKSalaries.csv")

os.chdir(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool")
dkng_file = shutil.copy("DKSalaries.csv", "DKSalaries_"+ contest + str(today)+".csv")
dkng_file = pd.read_csv(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool/DKSalaries.csv")
dkng_file.to_excel(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool/DKSalaries_"+ contest + str(today)+".xlsx", index=False)
os.remove("DKSalaries.csv")
os.remove("DKSalaries_"+ contest + str(today)+".csv")

In [119]:
## ETR Daily

# changes the working directory to your downloads folder
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/ETR_Data/DraftKings'
daily_file = "DK NBA Projections.csv"

shutil.move(downloads + "/"+ daily_file,
           data_loc + "/"+ daily_file)

os.chdir(data_loc)

df = pd.read_csv(daily_file)

df.to_excel("ETR_Daily_"+str(today)+".xlsx", index=False)
os.remove("DK NBA Projections.csv")

In [120]:
## Joins the DKNG and ETR Files to include DKNG Positions 
    ## ETR does not include G/F/UTIL

os.chdir(r"C:\Users\vchang\Documents\Fantasy Bball\DFS\PlayerPool Analysis")
etr = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\ETR_Data\DraftKings\ETR_Daily_' + str(today) + ".xlsx")
dkng = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\DKNG_Data\PlayerPool\DKSalaries_PlayerPool_' + str(today) + ".xlsx")

## TO DO - check if merge on only DKNG Roster Position Works
df = pd.merge(etr, 
              dkng[['Name','Roster Position', 'Name + ID']], 
              left_on='Player', 
              right_on='Name')

In [121]:
df.head()

,Player,Team,Opp,Minutes,DK Position,DK Salary,DK Points,DK Value,DK Ceiling,DK Own,DK Slate,Name,Roster Position,Name + ID
0,Lance Stephenson,IND,ATL,31.5,SG/SF,"3,200",33.4,14.6,42.3,59.8,MAIN,Lance Stephenson,SG/SF/F/G/UTIL,Lance Stephenson (21260371)
1,Justise Winslow,POR,ORL,29.0,PF,"3,000",26.1,8.1,34.1,38.6,MAIN,Justise Winslow,PF/F/UTIL,Justise Winslow (21261032)
2,Goga Bitadze,IND,ATL,29.0,C,"4,200",30.2,7.4,37.4,36.8,MAIN,Goga Bitadze,C/UTIL,Goga Bitadze (21260083)
3,Keifer Sykes,IND,ATL,33.0,PG,"3,900",28.4,6.8,36.7,14.4,MAIN,Keifer Sykes,PG/G/UTIL,Keifer Sykes (21260169)
4,Brandon Ingram,NOP,HOU,35.5,SG/SF,"9,200",49.4,6.6,57.4,38.4,MAIN,Brandon Ingram,SG/SF/F/G/UTIL,Brandon Ingram (21259671)


In [122]:
## Unnests Roster Positions into each Row (explode in Python) ##


## Update Salary to INT 
df['DK Salary'] = df['DK Salary'].str.replace(",","").astype(float)

df['Roster Position'] = df['Roster Position'].astype(str)
df['Roster Position'] = df['Roster Position'].str.split('/')

df.head()

,Player,Team,Opp,Minutes,DK Position,DK Salary,DK Points,DK Value,DK Ceiling,DK Own,DK Slate,Name,Roster Position,Name + ID
0,Lance Stephenson,IND,ATL,31.5,SG/SF,3200.0,33.4,14.6,42.3,59.8,MAIN,Lance Stephenson,"[SG, SF, F, G, UTIL]",Lance Stephenson (21260371)
1,Justise Winslow,POR,ORL,29.0,PF,3000.0,26.1,8.1,34.1,38.6,MAIN,Justise Winslow,"[PF, F, UTIL]",Justise Winslow (21261032)
2,Goga Bitadze,IND,ATL,29.0,C,4200.0,30.2,7.4,37.4,36.8,MAIN,Goga Bitadze,"[C, UTIL]",Goga Bitadze (21260083)
3,Keifer Sykes,IND,ATL,33.0,PG,3900.0,28.4,6.8,36.7,14.4,MAIN,Keifer Sykes,"[PG, G, UTIL]",Keifer Sykes (21260169)
4,Brandon Ingram,NOP,HOU,35.5,SG/SF,9200.0,49.4,6.6,57.4,38.4,MAIN,Brandon Ingram,"[SG, SF, F, G, UTIL]",Brandon Ingram (21259671)


In [123]:
df.to_excel("DKNG_final_PlayerPool_ETR_"+str(today)+".xlsx", index=False)
# os.system("DKNG_final_PlayerPool_ETR_"+str(today)+".xlsx")

# Create the Constraint Problem

Goal: Maximize DK Points

- Total Players = 8
- TotalSalary <= 50000
- TotalPosition_PG
- TotalPosition_SG
- TotalPosition_G
- TotalPosition_SF
- TotalPosition_PF
- TotalPosition_F
- TotalPosition_C
- TotalPosition_UTIL

In [124]:
prob = pulp.LpProblem('NBA_DFS_Daily', LpMaximize)

In [125]:
player_vars = [pulp.LpVariable(f'player_{row.Player}', cat='Binary') for row in df.itertuples()]

In [126]:
prob += pulp.lpSum(player_var for player_var in player_vars) == 8

In [127]:
# position constraints
def get_position_sum(player_vars, df, position):
    return pulp.lpSum([player_vars[i] * (position in df['Roster Position'].iloc[i]) for i in range(len(df))])
    

prob += get_position_sum(player_vars, df, 'PG') >= 1
prob += get_position_sum(player_vars, df, 'SG') >= 1
prob += get_position_sum(player_vars, df, 'G') >= 3
prob += get_position_sum(player_vars, df, 'SF') >= 1
prob += get_position_sum(player_vars, df, 'PF') >= 1
prob += get_position_sum(player_vars, df, 'F') >= 3
prob += get_position_sum(player_vars, df, 'C') >= 1
# prob += get_position_sum(player_vars, df, 'UTIL') >= 1

# prob += get_position_sum(player_vars, df, 'PG') <= 2
# prob += get_position_sum(player_vars, df, 'PG') >= 1
# prob += get_position_sum(player_vars, df, 'SG') <= 3
# prob += get_position_sum(player_vars, df, 'SG') >= 1
# prob += get_position_sum(player_vars, df, 'SF') <= 3
# prob += get_position_sum(player_vars, df, 'SF') >= 1
# prob += get_position_sum(player_vars, df, 'PF') <= 3
# prob += get_position_sum(player_vars, df, 'PF') >= 1
prob += get_position_sum(player_vars, df, 'C') <= 2
# prob += get_position_sum(player_vars, df, 'C') >= 1

In [128]:
# total salary constraint

prob += pulp.lpSum(df['DK Salary'].iloc[i] * player_vars[i] for i in range(len(df))) <= 50000

In [129]:
# finally, specify the goal
prob += pulp.lpSum([df['DK Points'].iloc[i] * player_vars[i] for i in range(len(df))])

In [130]:
# solve and print the status
prob.solve()
print(LpStatus[prob.status])

Optimal


In [131]:
# for each of the player variables, 
total_salary_used = 0
mean_AvgPointsPerGame = 0
for i in range(len(df)):
    if player_vars[i].value() == 1:
        row = df.iloc[i]
        print(row['Roster Position'], row.Player, row.Team, row['DK Salary'], row['DK Points'])
        total_salary_used += row['DK Salary']
        mean_AvgPointsPerGame += row['DK Points']
#mean_AvgPointsPerGame /= 9  # divide by total players in roster to get a mean
total_salary_used, mean_AvgPointsPerGame

['SG', 'SF', 'F', 'G', 'UTIL'] Lance Stephenson IND 3200.0 33.4
['PF', 'F', 'UTIL'] Justise Winslow POR 3000.0 26.1
['C', 'UTIL'] Goga Bitadze IND 4200.0 30.2
['PG', 'G', 'UTIL'] Keifer Sykes IND 3900.0 28.4
['SG', 'SF', 'F', 'G', 'UTIL'] Brandon Ingram NOP 9200.0 49.4
['PG', 'G', 'UTIL'] Dennis Smith Jr. POR 3100.0 23.5
['C', 'UTIL'] Nikola Jokic DEN 12600.0 61.2
['PG', 'SF', 'F', 'G', 'UTIL'] LeBron James LAL 10500.0 50.5


(49700.0, 302.7)